In [0]:
from pyspark.sql.functions import current_timestamp

tables = [
    "companies", "company_reviews", "employment_types", "industries",
    "job_benefits", "job_skills", "jobs", "locations", "salary_ranges", "skills"
]

for table_name in tables:
    print(f"Processando para camada Bronze: {table_name}")
    landing_path = f"/mnt/datalake/landing/{table_name}"
    bronze_path = f"/mnt/datalake/bronze/{table_name}"

    # Lê os dados da camada Landing
    df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(landing_path)

    # Adiciona metadados de ingestão
    df_with_metadata = df.withColumn("ingestion_date", current_timestamp())

    # Salva como tabela Delta na camada Bronze
    df_with_metadata.write.format("delta").mode("overwrite").save(bronze_path)

    print(f"Tabela {table_name} salva em formato Delta em {bronze_path}")